In [1]:
import sys
import polars as pl
from pathlib import Path

sys.path.insert(0, str(Path("../../").resolve()))
# Path relativo alla root del progetto (2 livelli sopra da strategies/example/)
data_dir = Path("../../database") 

from engine.datafeed import DataFeed, MarketData
from engine.engine import *
from engine.factor import *
from engine.plotting import *


In [2]:
# PRICES_PATH = str(project_root / 'data')
universe = ['SX5E']
start_date='2025-01-01'
end_date='2026-12-31'

# cfg = Config(universe=universe, start_date=start_date, end_date=end_date, frequency="eod")
feed = DataFeed(prices_base=data_dir)

In [ ]:
factors = [ATR(5),
            DonchianChannels(20)]

In [ ]:
researcher = Researcher(factors,
                        feed,
                        start_date, end_date,
                        frequency="eod",
                        tickers=universe)

In [6]:
fig = researcher.plot(ticker="SX5E")  # Plotta automaticamente tutti i fattori
fig.show()

# DataFeed DataFrame

In [9]:
# Carica i dati di mercato usando DataFeed
df = feed.get_market_data(start_date, end_date, 'eod', universe)
print(f"Shape: {df.shape}")
print(f"\nPrime righe:")
print(df.head())


Shape: (280, 10)

Prime righe:
shape: (5, 10)
┌────────────┬──────────┬────────┬─────────┬───┬─────────┬──────────────┬────────────────┬──────┐
│ date       ┆ time     ┆ ticker ┆ open    ┆ … ┆ close   ┆ volume       ┆ insertion_time ┆ type │
│ ---        ┆ ---      ┆ ---    ┆ ---     ┆   ┆ ---     ┆ ---          ┆ ---            ┆ ---  │
│ date       ┆ time     ┆ str    ┆ f64     ┆   ┆ f64     ┆ f64          ┆ datetime[μs]   ┆ str  │
╞════════════╪══════════╪════════╪═════════╪═══╪═════════╪══════════════╪════════════════╪══════╡
│ 2025-01-02 ┆ 01:00:00 ┆ SX5E   ┆ 4890.54 ┆ … ┆ 4917.88 ┆ 2.386088e8   ┆ 2026-02-03     ┆ eod  │
│            ┆          ┆        ┆         ┆   ┆         ┆              ┆ 16:40:00       ┆      │
│ 2025-01-03 ┆ 01:00:00 ┆ SX5E   ┆ 4915.54 ┆ … ┆ 4871.45 ┆ 2.06090877e8 ┆ 2026-02-03     ┆ eod  │
│            ┆          ┆        ┆         ┆   ┆         ┆              ┆ 16:40:00       ┆      │
│ 2025-01-06 ┆ 01:00:00 ┆ SX5E   ┆ 4893.32 ┆ … ┆ 4986.64 ┆ 2.62450529e8 

In [10]:
feed.get_data_excel(start_date, end_date, '1m', universe)

'c:\\Users\\S542279\\Desktop\\quanto\\strategies\\example\\SX5E_2025-01-01_2026-12-31_1m.xlsx'

In [11]:
# Crea MarketData dal DataFrame storico
market_data = MarketData.from_dataframe(df)
print(f"MarketData creato con {len(market_data.symbols())} simboli")
print(f"Current date: {market_data.current_date()}")
print(f"Current time: {market_data.current_time()}")


MarketData creato con 1 simboli
Current date: 2026-02-04
Current time: 01:00:00


In [12]:
# Esempi di accesso ai prezzi
print("Prezzo corrente:")
print(f"  SX5E close: ${market_data.price('SX5E', 'close'):.2f}")

print("\nPrezzo T-1 (1 bar fa):")
print(f"  SX5E close: ${market_data.price('SX5E', 'close', 1):.2f}")

print("\nUltimi 10 prezzi:")
last_10 = market_data.price('SX5E', 'close', slice(0, 10))
print(f"  Shape: {len(last_10)} valori")
print(f"  Ultimi 3: {last_10[-3:].to_list()}")


Prezzo corrente:
  SX5E close: $5970.47

Prezzo T-1 (1 bar fa):
  SX5E close: $5995.35

Ultimi 10 prezzi:
  Shape: 10 valori
  Ultimi 3: [6007.51, 5995.35, 5970.47]
